### 코드 출처 : https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 데이터 전처리

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [3]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [4]:
# Convert to list
data = df.content.values.tolist()

# 앞부분 이메일 제거
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# \s 제거
data = [re.sub('\s+', ' ', sent) for sent in data]

#작은따옴표 제거
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### spacy.load('en')에서 오류가 발생함.
## --> Anaconda Prompt를 관리자 권한으로 실행하고 python -m spacy download en   실행

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## 사전, 말뭉치 생성

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)]]


In [16]:
type(id2word)

gensim.corpora.dictionary.Dictionary

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('specs', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [18]:
# Build LDA model
# corpus - 말뭉치 list
# id2word - 단어와 그 단어의 빈도수를 묶은 튜플 리스트
# alpha, beta ==> 'auto' ==> 1/num_topics
# chunksize is the number of documents to be used in each training chunk.
# update_every determines how often the model parameters should be updated and passes is the total number of training passes.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.069*"air" + 0.038*"ohio_state" + 0.026*"window_manager" + 0.026*"lee" + '
  '0.025*"dos" + 0.019*"henry_spencer" + 0.018*"gate" + 0.018*"ltd" + '
  '0.017*"toolkit" + 0.017*"borland"'),
 (1,
  '0.040*"cpu" + 0.035*"docs" + 0.034*"logo" + 0.030*"patch" + 0.026*"tonight" '
  '+ 0.017*"phillie" + 0.004*"attain" + 0.001*"padre" + 0.000*"libxmu" + '
  '0.000*"instal"'),
 (2,
  '0.036*"use" + 0.031*"problem" + 0.020*"drive" + 0.019*"card" + '
  '0.018*"system" + 0.014*"bit" + 0.012*"work" + 0.011*"high" + 0.011*"power" '
  '+ 0.010*"standard"'),
 (3,
  '0.059*"hall" + 0.044*"astronomy" + 0.036*"dare" + 0.031*"troop" + '
  '0.028*"sunday" + 0.025*"libertarian" + 0.012*"gary_dare" + '
  '0.007*"celebrate" + 0.006*"phds" + 0.004*"souviens_gary"'),
 (4,
  '0.071*"tv" + 0.068*"insurance" + 0.046*"steven" + 0.036*"taxis" + '
  '0.016*"rent" + 0.009*"tocchet" + 0.006*"announcer" + 0.002*"baltimore" + '
  '0.000*"pts_pt" + 0.000*"pgh"'),
 (5,
  '0.092*"key" + 0.033*"encryption" + 0.033*"av

### => 0번 토픽에 할당된 top 10단어가 air, ohio_state, window_manager ... 등이고, 토픽 0의 각각의 weight이 앞의 소수인 것
###     => 할당된 토픽들의 키워드를 보면서, 각토픽이 어떠한 주제를 담고 있는지 유추가 가능함. 

## Perplexity Score : 문서 할당의 최적의 토픽을 찾기 위한 지표

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.744777955527061

Coherence Score:  0.47803382593678184


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\EdwinPark\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.377297 -0.106883       1        1  37.310429
14     0.373026 -0.122231       2        1  19.916618
6      0.273791 -0.040221       3        1  10.196003
2      0.266417 -0.031392       4        1   9.843257
16     0.213591  0.017718       5        1   6.511382
8      0.123589  0.459726       6        1   5.911445
18     0.055022  0.047526       7        1   1.999234
11    -0.035930  0.048424       8        1   1.722844
15     0.015165 -0.012104       9        1   1.647999
5     -0.026341 -0.020174      10        1   1.353577
7     -0.103488  0.003359      11        1   1.200269
9     -0.110107  0.000161      12        1   0.790694
17    -0.151201 -0.020351      13        1   0.462568
13    -0.167203 -0.024554      14        1   0.369179
0     -0.176239 -0.030594      15        1   0.291649
19    -0.178647 -0.032694      16        1   0.131527
4     -0.184058 -0.033108      17        1   0.127129
3     -0.183474 -0.032729      18        1   0.123579
1     -0.187509 -0.034061      19        1   0.087175
12    -0.193701 -0.035819      20        1   0.003441, topic_info=      Category          Freq             Term        Total  loglift  logprob
term                                                                        
4104   Default  68545.000000               ax  68545.00000  30.0000  30.0000
30     Default  16398.000000     organization  16398.00000  29.0000  29.0000
21     Default  18371.000000             line  18371.00000  28.0000  28.0000
48     Default   5366.000000             year   5366.00000  27.0000  27.0000
80     Default  24754.000000              not  24754.00000  26.0000  26.0000
3681   Default   4839.000000              max   4839.00000  25.0000  25.0000
316    Default   7181.000000              use   7181.00000  24.0000  24.0000
672    Default   4371.000000            right   4371.00000  23.0000  23.0000
1882   Default   2743.000000            space   2743.00000  22.0000  22.0000
668    Default   4772.000000          problem   4772.00000  21.0000  21.0000
2699   Default   2577.000000             game   2577.00000  20.0000  20.0000
53     Default  11164.000000          article  11164.00000  19.0000  19.0000
116    Default  10287.000000              get  10287.00000  18.0000  18.0000
322    Default  14172.000000            write  14172.00000  17.0000  17.0000
410    Default   7222.000000           people   7222.00000  16.0000  16.0000
301    Default  10245.000000              say  10245.00000  15.0000  15.0000
561    Default   2447.000000             team   2447.00000  14.0000  14.0000
201    Default   5219.000000           system   5219.00000  13.0000  13.0000
749    Default   2402.000000              law   2402.00000  12.0000  12.0000
63     Default  15563.000000               do  15563.00000  11.0000  11.0000
336    Default   3413.000000            state   3413.00000  10.0000  10.0000
747    Default   1972.000000              key   1972.00000   9.0000   9.0000
241    Default  12208.000000               be  12208.00000   8.0000   8.0000
210    Default   3611.000000           window   3611.00000   7.0000   7.0000
149    Default  13731.000000            would  13731.00000   6.0000   6.0000
152    Default   2061.000000        available   2061.00000   5.0000   5.0000
1345   Default   2331.000000       government   2331.00000   4.0000   4.0000
2381   Default   3240.000000      information   3240.00000   3.0000   3.0000
260    Default   3085.000000            drive   3085.00000   2.0000   2.0000
2159   Default   1723.000000              gun   1723.00000   1.0000   1.0000
...        ...           ...              ...          ...      ...      ...
25487  Topic20      0.000768            pspro      1.04623   3.0602 -11.1705
23670  Topic20      0.000768           helpen      1.04623   3.0602 -11.1705
23664  Topic20      0.000768        fakefully      1.04623   3.0602 -11.17